In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-08-31 12:21:28--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-08-31 12:21:28--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8539d59ae6b57fcfb0e8901733.dl.dropboxusercontent.com/cd/0/inline/BVQ2NHOllwh9UtbX0Al86kx5fP9gdPHFLQyrb5OkAaAMArQsa1hC8-oC2iksRvWXbDTJY0up-4_mIJHMyvC-vJSIJAsfEU4bphPL2fcOHmOerzQWxC6yUlNbEOi5RNZ9x2yiy8phg1i7I1sY-nSpNfYJ/file# [following]
--2021-08-31 12:21:28--  https://uc8539d59ae6b57fcfb0e8901733.dl.dropboxusercontent.com/cd/0/inline/BVQ2NHOllwh9UtbX0Al86kx5fP9gdPHFLQyrb5OkAaAMArQsa1h

In [ ]:
# Let's unzip the file
!unzip -q "/content/movie-reviews-dataset.zip"

In [ ]:
from tensorflow.keras.preprocessing import  text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional

In [ ]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, "<br />", " "), label),
  )

In [ ]:
train_data = prepareData("/content/movie-reviews-dataset/train")
test_data = prepareData("/content/movie-reviews-dataset/test")

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'*spoliers* do not read any further if you haven\'t seen this movie  Picking up after the depressing "Phantasm II" ended; The Tall Man kidnaps Mike, while Reggie and new kid Tim spend most of the rest of the movie trying to get him back, and not end up as slaves on the Tall Man\'s "Red Planet". This one gets really silly: the trio of thieves in the bright pink hearse were only there for comic relief, and the black karate chick (can\'t remember her name) was so irritating I couldn\'t wait to see if the Tall Man killed her character. This one sets it up to almost look like it was going to be Tim\'s character, who comes in late and ultimately is the hero, but ... it doesn\'t quite work out that way. By the very end of this story Liz is beheaded by the killer midgets, Mike gets a silver sphere implanted into his skull, the spheres get Reggie and the dwarfs get Tim, and there\'s no one left to stop The Tall

In [ ]:
model =Sequential()

In [ ]:
model.add(Input(shape=(1,), dtype="string"))

In [ ]:
max_tokens= 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens = max_tokens,
    output_mode = 'int',
    output_sequence_length = max_len,
)

In [ ]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [ ]:
model.add(Embedding(max_tokens + 1, 128))

model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 114s 140ms/step - loss: 0.5085 - accuracy: 0.7428
Epoch 2/10
782/782 [==============================] - 112s 143ms/step - loss: 0.4249 - accuracy: 0.8041
Epoch 3/10
782/782 [==============================] - 107s 136ms/step - loss: 0.3929 - accuracy: 0.8194
Epoch 4/10
782/782 [==============================] - 107s 136ms/step - loss: 0.3642 - accuracy: 0.8356
Epoch 5/10
782/782 [==============================] - 107s 136ms/step - loss: 0.3338 - accuracy: 0.8508
Epoch 6/10
782/782 [==============================] - 107s 136ms/step - loss: 0.3051 - accuracy: 0.8644
Epoch 7/10
782/782 [==============================] - 107s 137ms/step - loss: 0.2799 - accuracy: 0.8754
Epoch 8/10
782/782 [==============================] - 107s 137ms/step - loss: 0.2598 - accuracy: 0.8843
Epoch 9/10
782/782 [==============================] - 107s 137ms/step - loss: 0.2348 - accuracy: 0.8955
Epoch 10/10
782/782 [==============================] - 107s 136m

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 32s 40ms/step - loss: 0.8023 - accuracy: 0.7592


[0.8023123145103455, 0.7591999769210815]

In [ ]:
text = "This movis is bad"

In [ ]:
model.predict([text]) # 0 = Negative review, 1 = positive review

array([[0.05701149]], dtype=float32)

In [ ]:
text2 = "This movis is good"

In [ ]:
model.predict([text2]) # 0 = Negative review, 1 = positive review

array([[0.9085814]], dtype=float32)